In [41]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from sklearn.model_selection import KFold
import tensorflow as tf
import numpy as np

#### Import model, training function 

In [7]:
from annsa.load_dataset import dataset_to_spectrakeys
from annsa.model_classes import build_dnn_model, compile_model
from annsa.load_pretrained_network import load_features

Using TensorFlow backend.


## Dataset Construction

#### Load dataset

In [55]:
dataset = np.load(os.path.join('dataset_generation',
                               'uenrichment_dataset_100000.npy'))
training_spectra = np.array(dataset.item()['spectra'], dtype='float64')
all_keys = np.array(dataset.item()['keys'])

## Model Construction

In [74]:
architecture_id = os.path.join('..',
                               'source-interdiction',
                               'final_training_notebooks',
                               'final-models',
                               'learningcurve-dnn-full-final-features',
                              )

In [75]:
model_features = load_features(architecture_id)
model_features.input_dim = 1024
model_features.dropout_rate = model_features.dropout_probability
model_features.loss = tf.keras.losses.mean_squared_error
model_features.optimizer = tf.keras.optimizers.Adam
model_features.output_function = tf.nn.sigmoid
model_features.output_size = 1
model_features.metrics = ['mse']
model_features.learning_rate = model_features.learining_rate

# Train network

In [76]:
num_epochs = 200
patience = 10

model_id_save_as = 'dnn-full'

earlystop_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_mean_squared_error',
    patience=patience,
    mode='min',
    min_delta=0.01,
    restore_best_weights=True)

## Scale input data

In [77]:
training_spectra_scaled = model_features.scaler.transform(training_spectra)

# Train

In [78]:
kf = KFold(n_splits=5)

k = 0
for train_index, validation_index in kf.split(training_spectra_scaled):
    model = compile_model(
        build_dnn_model,
        model_features)

    csv_logger = tf.keras.callbacks.CSVLogger(
        os.path.join('final-models-keras/',
        model_id_save_as+'_fold' + str(k) + '.log')

    output = model.fit(
        x=training_spectra_scaled[train_index],
        y=all_keys[train_index],
        epochs=num_epochs,
        verbose=1,
        validation_data=(training_spectra_scaled[validation_index],
                         all_keys[validation_index]),
        shuffle=True,
        callbacks=[earlystop_callback, csv_logger],
    )
    model.save(
        os.path.join('final-models-keras',
                     model_id_save_as + '_fold' + str(k) + '.hdf5')
    k += 1

Train on 80000 samples, validate on 20000 samples
Epoch 1/200
80000/80000 [==============================] - 11s 140us/sample - loss: 3.9937 - mean_squared_error: 0.1343 - val_loss: 0.2061 - val_mean_squared_error: 0.0615
Epoch 2/200
80000/80000 [==============================] - 11s 132us/sample - loss: 0.1223 - mean_squared_error: 0.0583 - val_loss: 0.0743 - val_mean_squared_error: 0.0487
Epoch 3/200
80000/80000 [==============================] - 11s 133us/sample - loss: 0.0705 - mean_squared_error: 0.0537 - val_loss: 0.0587 - val_mean_squared_error: 0.0465
Epoch 4/200
80000/80000 [==============================] - 11s 133us/sample - loss: 0.0626 - mean_squared_error: 0.0521 - val_loss: 0.0569 - val_mean_squared_error: 0.0476
Epoch 5/200
80000/80000 [==============================] - 11s 133us/sample - loss: 0.0591 - mean_squared_error: 0.0505 - val_loss: 0.0707 - val_mean_squared_error: 0.0627
Epoch 6/200
80000/80000 [==============================] - 11s 134us/sample - loss: 0.0564

80000/80000 [==============================] - 9s 116us/sample - loss: 0.0483 - mean_squared_error: 0.0438 - val_loss: 0.0422 - val_mean_squared_error: 0.0375
Epoch 21/200
80000/80000 [==============================] - 10s 126us/sample - loss: 0.0484 - mean_squared_error: 0.0439 - val_loss: 0.0437 - val_mean_squared_error: 0.0392
